In [1]:
import json
import pennylane as qml
import pennylane.numpy as np

In [3]:
def U_psi(theta):
    """
    Quantum function that generates |psi>, Zenda's state wants to send to Reece.

    Args:
        theta (float): Parameter that generates the state.

    """
    qml.Hadamard(wires = 0)
    qml.CRX(theta, wires = [0,1])
    qml.CRZ(theta, wires = [0,1])

def is_unsafe(alpha, beta, epsilon):
    """
    Boolean function that we will use to know if a set of parameters is unsafe.

    Args:
        alpha (float): parameter used to encode the state.
        beta (float): parameter used to encode the state.
        epsilon (float): unsafe-tolerance.

    Returns:
        (bool): 'True' if alpha and beta are epsilon-unsafe coefficients. 'False' in the other case.

    """

    from numpy import cos, sin, exp

    # NOTE: I found this expression analytically by calculating the inner product between |psi(theta)> and  U(alpha)U(beta)|psi(theta)>
    def f(theta, alpha, beta):
            # here is where I can put the function I have. Either keep as it is or convert using 
            # cos theta/2 = x,  sin theta/2 = np.sqrt(1-x**2)
            # e^(i\theta/2) = cos(theta/2) + i sin(theta/2) =  x + 1j*np.sqrt(1-x**2)
            # e^(-i\theta/2) = cos(theta/2) - i sin(theta/2) =  x - 1j*np.sqrt(1-x**2)
            r = np.exp(-1j*alpha)*np.cos(beta/2)**2 - (1j*np.cos(theta/2)*np.exp(-1j*theta/2)*np.sin(beta)*0.5) + 1j*np.sin(theta/2)*np.sin(beta/2)**2*np.exp(1j*theta/2)*np.exp(1j*alpha) \
                + np.cos(theta/2)*np.exp(1j*theta/2)*(np.cos(theta/2)*np.cos(beta/2)**2*np.exp(-1j*theta/2)-(1j*np.exp(-1j*alpha)*np.sin(beta)*0.5) - (0.5*np.sin(theta/2)*np.sin(beta)*np.exp(1j*theta/2)*np.exp(1j*alpha)))\
                - 1j*np.sin(theta/2)*np.exp(-1j*theta/2)*np.exp(-1j*alpha)*(np.sin(beta/2)**2*np.exp(-1j*alpha) + (0.5j*np.cos(theta/2)*np.sin(beta)*np.exp(-1j*theta/2)) + 1j*np.sin(theta/2)*np.cos(beta/2)**2*np.exp(1j*theta/2)*np.exp(1j*alpha))
            return abs(r/2)**2

    def cost(theta, alpha, beta, epsilon):
        return 1 - epsilon - f(theta, alpha, beta)
   
    def minimize(cost, x0, alpha, beta, epsilon, conv_tol=1e-08, step_size=0.2, max_iterations=500, progress=False):
        # NOTE: had to create own optimizitation since importing was not allowed. else could have used scipy.minimize
        opt = qml.AdamOptimizer(stepsize=step_size)
        #opt = qml.GradientDescentOptimizer(stepsize=step_size)
        #opt = qml.QNGOptimizer(stepsize=step_size, approx="block-diag")
        theta = np.array(x0, requires_grad=True)
        # store the values of the cost function
        energy = [cost(theta, alpha, beta, epsilon)]
        # store the values of the circuit parameter
        params = [theta]

        for n in range(max_iterations):
            theta, prev_energy = opt.step_and_cost(cost, theta, alpha=alpha, beta=beta, epsilon=epsilon) #define the other kwargs at the end
            energy.append(cost(theta, alpha, beta, epsilon))
            params.append(theta)

            conv = np.abs(energy[-1] - prev_energy)
            if progress:
                print(f"Step = {n},  Energy = {energy[-1]:.8f} Ha")
            if conv <= conv_tol:
                break
        status = f'finished in {n} iterations' 
        
        return params[-1], energy[-1], status

    #from scipy.optimize import minimize: NOT ALLOWED
    x0 = np.pi # initial guess
    
    #res = minimize(cost, x0, args=(alpha,beta,epsilon), method='Nelder-Mead', tol=1e-6)
    x, val, status = minimize(cost, x0, alpha, beta, epsilon)
    #this checks that solution and far enough from zero (0.02) since otherwise it can be negative due to numerical noise
    if val <= 0 and abs(val)>0.02:
        return True
    else:
        return False

    # Put your code here #
    

# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:
    ins = json.loads(test_case_input)
    output = is_unsafe(*ins)
    return str(output)

def check(solution_output: str, expected_output: str) -> None:
    
    def bool_to_int(string):
        if string == "True":
            return 1
        return 0

    solution_output = bool_to_int(solution_output)
    expected_output = bool_to_int(expected_output)
    assert solution_output == expected_output, "The solution is not correct."


test_cases = [['[0.1, 0.2, 0.3]', 'True'], ['[1.1, 1.2, 0.3]', 'False'], ['[1.1, 1.2, 0.4]', 'True'], ['[0.5, 1.9, 0.7]', 'True']]

for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[0.1, 0.2, 0.3]'...
Correct!
Running test case 1 with input '[1.1, 1.2, 0.3]'...
Correct!
Running test case 2 with input '[1.1, 1.2, 0.4]'...
Correct!
Running test case 3 with input '[0.5, 1.9, 0.7]'...
Correct!


# Idea:
to find if a solution $\theta$ exists such that 

$$f(\theta, \alpha, \beta) \geq 1-\epsilon$$

we can look at 
$$1 - \epsilon - f(\theta, \alpha, \beta) \leq 0$$

if this inequality is satisfied, it means that the initial inequality was satisfied i.e. $\alpha$ and $\beta$ are $\epsilon$-unsafe

Then, we minimize the left hand side wrt $\theta$ and the min is negative (within tolerance) we return `True`, else `False`

-----
# <center> Tests

In [79]:

def f(theta, alpha, beta):
        # here is where I can put the function I have. Either keep as it is or convert using 
        # cos theta/2 = x,  sin theta/2 = np.sqrt(1-x**2)
        # e^(i\theta/2) = np.cos(theta/2) + i np.sin(theta/2) =  x + 1j*np.sqrt(1-x**2)
        # e^(-i\theta/2) = np.cos(theta/2) - i np.sin(theta/2) =  x - 1j*np.sqrt(1-x**2)
        r = np.exp(-1j*alpha)*np.cos(beta/2)**2 - (1j*np.cos(theta/2)*np.exp(-1j*theta/2)*np.sin(beta)*0.5) + 1j*np.sin(theta/2)*np.sin(beta/2)**2*np.exp(1j*theta/2)*np.exp(1j*alpha) \
                + np.cos(theta/2)*np.exp(1j*theta/2)*(np.cos(theta/2)*np.cos(beta/2)**2*np.exp(-1j*theta/2)-(1j*np.exp(-1j*alpha)*np.sin(beta)*0.5) - (0.5*np.sin(theta/2)*np.sin(beta)*np.exp(1j*theta/2)*np.exp(1j*alpha)))\
                - 1j*np.sin(theta/2)*np.exp(-1j*theta/2)*np.exp(-1j*alpha)*(np.sin(beta/2)**2*np.exp(-1j*alpha) + (0.5j*np.cos(theta/2)*np.sin(beta)*np.exp(-1j*theta/2)) + 1j*np.sin(theta/2)*np.cos(beta/2)**2*np.exp(1j*theta/2)*np.exp(1j*alpha))
        return abs(r/2)**2 

def cost(theta, alpha, beta, epsilon):
        return 1 - epsilon - f(theta, alpha, beta)

In [38]:
f(1, 1, 1)

0.6867982895270761

In [112]:
def minimize(cost, x0, alpha, beta, epsilon, conv_tol=1e-08, step_size=0.2, max_iterations=500, progress=False):
    opt = qml.AdamOptimizer(stepsize=step_size)
    #opt = qml.GradientDescentOptimizer(stepsize=step_size)
    #opt = qml.QNGOptimizer(stepsize=step_size, approx="block-diag")
    theta = np.array(x0, requires_grad=True)
    # store the values of the cost function
    energy = [cost(theta, alpha, beta, epsilon)]
    # store the values of the circuit parameter
    params = [theta]

    for n in range(max_iterations):
        theta, prev_energy = opt.step_and_cost(cost, theta, alpha=alpha, beta=beta, epsilon=epsilon)
        energy.append(cost(theta, alpha, beta, epsilon))
        params.append(theta)

        conv = np.abs(energy[-1] - prev_energy)
        if progress:
            print(f"Step = {n},  Energy = {energy[-1]:.8f} Ha")
        if conv <= conv_tol:
            break
    status = f'finished in {n} iterations' 
    
    return params, energy, status

In [113]:
params, energy, status = minimize(cost, np.pi, 0.5, 1.9, 0.7)

In [114]:
print(params[-1])
print(energy[-1])
print(status)

0.8347076843874878
-0.06527307439003349
finished in 143 iterations


In [71]:

def is_unsafe(alpha, beta, epsilon):
    """
    Boolean function that we will use to know if a set of parameters is unsafe.

    Args:
        alpha (float): parameter used to encode the state.
        beta (float): parameter used to encode the state.
        epsilon (float): unsafe-tolerance.

    Returns:
        (bool): 'True' if alpha and beta are epsilon-unsafe coefficients. 'False' in the other case.

    """
    from scipy.optimize import minimize
    x0 = np.pi
    #bnds = [0,2*np.pi]
    res = minimize(cost, x0, args=(alpha,beta,epsilon), method='Nelder-Mead',tol=1e-6)
    print(res.message)
    print(res.fun)
    print(res.x)
    if res.fun < 0 and abs(res.fun)>0.02:
        return True
    else:
        return False


In [108]:
cost(5.38198478, 0.1, 0.2, 0.3)

-0.28925716090144493

In [109]:
cost(7.10332537876704, 0.1, 0.2, 0.3)

-0.2894230213018424

In [72]:
is_unsafe(0.1, 0.2, 0.3)

Optimization terminated successfully.
-0.2892571609014454
[5.38198478]


True

In [73]:
is_unsafe(0.5, 1.9, 0.7)

Optimization terminated successfully.
-0.0652730921542814
[0.83440466]


True

In [74]:
is_unsafe(1.1, 1.2, 0.3)

Optimization terminated successfully.
-0.013397098820838726
[4.38120912]


False